#Bibliotecas 


In [ ]:
import pandas as pd
from google.cloud import storage
import os

#Importando DataFrame

In [ ]:
df = pd.read_csv(r'https://storage.googleapis.com/projetofinalgrupo8/saida/data_drop_world.csv')
#df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)

In [ ]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,trending_date,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,country,json_title
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12T00:00:00Z,263835,85095,487,4500,False,False,BR,People & Blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12T00:00:00Z,6000070,714310,15176,31040,False,False,BR,Music
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10T14:59:00Z,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,2020-08-12T00:00:00Z,2296748,39761,5484,0,True,False,BR,People & Blogs
3,gBjox7vn3-g,Contos de Runeterra: Targon | A Estrada Tortuosa,2020-08-11T15:00:09Z,UC6Xqz2pm50gDCORYztqhDpg,League of Legends BR,2020-08-12T00:00:00Z,300510,46222,242,2748,False,False,BR,Gaming
4,npoUGx7UW7o,Entrevista com Thammy Miranda | The Noite (10/...,2020-08-11T20:04:02Z,UCEWOoncsrmirqnFqxer9lmA,The Noite com Danilo Gentili,2020-08-12T00:00:00Z,327235,22059,3972,2751,False,False,BR,Comedy


#Verificando dimensão do DataFrame

In [ ]:
df.shape

(1021622, 14)

#Criando listas para renomear colunas

In [ ]:
col_old = ['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'trending_date', 'view_count', 'likes', 'dislikes', 'comment_count',
       'comments_disabled', 'ratings_disabled', 'country', 'json_title']

col_new = ['id_video', 'titulo_video', 'publicado_em', 'id_canal', 'nome_canal',
           'data_destaque', 'cont_visualizacao', 'curtidas', 'nao_curtidas', 'cont_comentarios',
           'comentarios_desabilitados', 'curtidas_desabilitadas', 'pais', 'categoria']

#Backup

In [ ]:
df2 = df.copy()

#Renomeando colunas

In [ ]:
df2.columns = col_new

In [ ]:
df2.head()

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12T00:00:00Z,263835,85095,487,4500,False,False,BR,People & Blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12T00:00:00Z,6000070,714310,15176,31040,False,False,BR,Music
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10T14:59:00Z,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,2020-08-12T00:00:00Z,2296748,39761,5484,0,True,False,BR,People & Blogs
3,gBjox7vn3-g,Contos de Runeterra: Targon | A Estrada Tortuosa,2020-08-11T15:00:09Z,UC6Xqz2pm50gDCORYztqhDpg,League of Legends BR,2020-08-12T00:00:00Z,300510,46222,242,2748,False,False,BR,Gaming
4,npoUGx7UW7o,Entrevista com Thammy Miranda | The Noite (10/...,2020-08-11T20:04:02Z,UCEWOoncsrmirqnFqxer9lmA,The Noite com Danilo Gentili,2020-08-12T00:00:00Z,327235,22059,3972,2751,False,False,BR,Comedy


#Vericando tipos de categorias

In [ ]:
df2['categoria'].unique()

array(['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals', nan], dtype=object)

#Criando listas para renomear as categorias

In [ ]:
categoria_antiga = ['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals']

categoria_nova = ['pessoas_e_blogs', 'musica', 'jogos', 'comedia', 'esportes',
                  'entretenimento', 'educacao', 'automoveis', 'como_faz_e_estilos',
                  'noticias_e_politicas', 'ciencias_e_tecnologia', 'filme_e_animacao',
                  'viagens_e_eventos', 'animais']

#Renomeando categorias

In [ ]:
df2['categoria'] = df2['categoria'].replace(categoria_antiga, categoria_nova)

#Verificando números de dados nulos

In [ ]:
df2.isna().sum()

id_video                        0
titulo_video                    0
publicado_em                    0
id_canal                        0
nome_canal                      1
data_destaque                   0
cont_visualizacao               0
curtidas                        0
nao_curtidas                    0
cont_comentarios                0
comentarios_desabilitados       0
curtidas_desabilitadas          0
pais                            0
categoria                    2951
dtype: int64

#Tratando os valores nulos da coluna "Categoria"

In [ ]:
import numpy as np

In [ ]:
categoria_antiga = ['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals', np.nan]

categoria_nova = ['pessoas_e_blogs', 'musica', 'jogos', 'comedia', 'esportes',
                  'entretenimento', 'educacao', 'automoveis', 'como_faz_e_estilos',
                  'noticias_e_politicas', 'ciencias_e_tecnologia', 'filme_e_animacao',
                  'viagens_e_eventos', 'animais', 'outros']

In [ ]:
df2['categoria'] = df2['categoria'].replace(categoria_antiga, categoria_nova)

In [ ]:
df2.isna().sum()

id_video                     0
titulo_video                 0
publicado_em                 0
id_canal                     0
nome_canal                   1
data_destaque                0
cont_visualizacao            0
curtidas                     0
nao_curtidas                 0
cont_comentarios             0
comentarios_desabilitados    0
curtidas_desabilitadas       0
pais                         0
categoria                    0
dtype: int64

#Verificando nomes de canais com valor nulo

In [ ]:
df2[df2['nome_canal'].isnull()]

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
499155,9b9MovPPewk,Kala Official Teaser | Tovino Thomas | Rohith ...,2021-01-21T12:30:29Z,UCDQt0y-FCJLRdwhNLutPFZA,NaN,2021-01-23T00:00:00Z,1008638,72772,1451,3023,False,False,IN,pessoas_e_blogs


#Removendo linha com nome de canal nulo

In [ ]:
df2.drop(499155, inplace = True)

In [ ]:
df2.isna().sum()

id_video                     0
titulo_video                 0
publicado_em                 0
id_canal                     0
nome_canal                   0
data_destaque                0
cont_visualizacao            0
curtidas                     0
nao_curtidas                 0
cont_comentarios             0
comentarios_desabilitados    0
curtidas_desabilitadas       0
pais                         0
categoria                    0
dtype: int64

In [ ]:
df2.head(3)

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12T00:00:00Z,263835,85095,487,4500,False,False,BR,pessoas_e_blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12T00:00:00Z,6000070,714310,15176,31040,False,False,BR,musica
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10T14:59:00Z,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,2020-08-12T00:00:00Z,2296748,39761,5484,0,True,False,BR,pessoas_e_blogs


#Verificando os tipos de dados do DataFrame

In [ ]:
df2.dtypes

id_video                     object
titulo_video                 object
publicado_em                 object
id_canal                     object
nome_canal                   object
data_destaque                object
cont_visualizacao             int64
curtidas                      int64
nao_curtidas                  int64
cont_comentarios              int64
comentarios_desabilitados      bool
curtidas_desabilitadas         bool
pais                         object
categoria                    object
dtype: object

#Formatando datetime

In [ ]:
df2['publicado_em'] = pd.to_datetime(df2['publicado_em']).dt.strftime("%Y-%m-%d %H:%M:%S")
df2['publicado_em'] = pd.to_datetime(df2['publicado_em'])

In [ ]:
df2['data_destaque'] = pd.to_datetime(df2['data_destaque']).dt.strftime("%Y-%m-%d")
df2['data_destaque'] = pd.to_datetime(df2['data_destaque'])

In [ ]:
df2.sample()

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
241849,aZBOJdM1jyU,Eins noch bis Müller: Lewa-Hattrick bei Meiste...,2021-05-09 22:00:00,UClBFnQJMlinWDCvfSXj60CA,DAZN Bundesliga,2021-05-14,1289245,42040,922,1698,False,False,DE,esportes


#Informações gerais sobre os tipo de dados e quantidades de não nulos

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021621 entries, 0 to 1021621
Data columns (total 14 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   id_video                   1021621 non-null  object        
 1   titulo_video               1021621 non-null  object        
 2   publicado_em               1021621 non-null  datetime64[ns]
 3   id_canal                   1021621 non-null  object        
 4   nome_canal                 1021621 non-null  object        
 5   data_destaque              1021621 non-null  datetime64[ns]
 6   cont_visualizacao          1021621 non-null  int64         
 7   curtidas                   1021621 non-null  int64         
 8   nao_curtidas               1021621 non-null  int64         
 9   cont_comentarios           1021621 non-null  int64         
 10  comentarios_desabilitados  1021621 non-null  bool          
 11  curtidas_desabilitadas     1021621 no

In [ ]:
df2

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11 22:21:49,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12,263835,85095,487,4500,False,False,BR,pessoas_e_blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11 15:00:13,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12,6000070,714310,15176,31040,False,False,BR,musica
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10 14:59:00,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,2020-08-12,2296748,39761,5484,0,True,False,BR,pessoas_e_blogs
3,gBjox7vn3-g,Contos de Runeterra: Targon | A Estrada Tortuosa,2020-08-11 15:00:09,UC6Xqz2pm50gDCORYztqhDpg,League of Legends BR,2020-08-12,300510,46222,242,2748,False,False,BR,jogos
4,npoUGx7UW7o,Entrevista com Thammy Miranda | The Noite (10/...,2020-08-11 20:04:02,UCEWOoncsrmirqnFqxer9lmA,The Noite com Danilo Gentili,2020-08-12,327235,22059,3972,2751,False,False,BR,comedia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021617,I-Bz1uGMcdw,Ресторан СЕВЕРНОЙ кухни / 3 (ТРИ!) награды Миш...,2021-11-16 09:59:59,UCxF6ylvxiQV5m-7Rawr3W7g,Макс Брандт,2021-11-17,271230,33841,506,828,False,False,RU,entretenimento
1021618,DoyhJP_e7JQ,Brawl Stars: Brawl Talk - It's NOT Club Wars! 😱,2021-11-13 16:00:10,UCooVYzDxdwTtGYAkcPmOgOw,Brawl Stars,2021-11-17,9810853,560362,10232,32452,False,False,RU,jogos
1021619,1xH8kzEzMuM,Հայլուր 20:30 Թշնամու ներխուժում. հրավիրվել է ...,2021-11-16 17:45:42,UCvhOxd4DL1kUZLOPJV-K-aA,5 TV Channel,2021-11-17,45658,325,64,0,False,False,RU,noticias_e_politicas
1021620,CgVhSYkL7lM,Изобретение кубанского пенсионера заинтересова...,2021-11-16 06:15:00,UC_IEcnNeHc_bwd92Ber-lew,Россия 24,2021-11-17,390957,11339,182,1654,False,False,RU,noticias_e_politicas


#Exportando DataFrame para o GCP

In [ ]:
from google.cloud import storage
import os
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')
    
bucket.blob('saida/data_tratado_pandas.csv').upload_from_string(df2.to_csv(index=False), 'text/csv')